In [1]:
import pandas as pd
import numpy as np
import re
from datetime import date
import matplotlib.pyplot as plt

In [2]:
# read in the scraped data file
df = pd.read_csv("glassdoor_jobs.csv", names = ['index', 'job_title', 'salary', 'job_description',  'company_rating', 'company_name', 'location', 'headquarters', 'company_size', 'founded', 'ownership_type', 'industry', 'sector', 'revenue', 'competitors' ])

# drop entries with no salary data
df = df[df['salary']!= "-1"]

# drop the first row and first column from the dataset
df = df.iloc[1:, 1:]

df.head()


,job_title,salary,job_description,company_rating,company_name,location,headquarters,company_size,founded,ownership_type,industry,sector,revenue,competitors
1,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
2,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
3,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
4,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
5,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [3]:
# view the data type of each column of the dataframe
df.dtypes

job_title          object
salary             object
job_description    object
company_rating     object
company_name       object
location           object
headquarters       object
company_size       object
founded            object
ownership_type     object
industry           object
sector             object
revenue            object
competitors        object
dtype: object

In [4]:
# convert the founded column to integer
df['founded'] = df['founded'].apply(int)
# convert the company rating column to a float
df['company_rating'] = df['company_rating'].apply(float)

# replace -1 values with NaN (missing value)
df = df.replace(to_replace = -1, value = np.nan)
# look at the number of missing values in each column of the dataframe
df.isnull().sum()

job_title           0
salary              0
job_description     0
company_rating     11
company_name        0
location            0
headquarters        0
company_size        0
founded            50
ownership_type      0
industry            0
sector              0
revenue             0
competitors         0
dtype: int64

In [5]:
# impute missing values for company rating and founded using the mean of the respective column
mean_rating = df['company_rating'].mean()
df['company_rating'] = df['company_rating'].replace(to_replace = np.nan, value = mean_rating)

mean_founded = df['founded'].mean()
df['founded'] = df['founded'].replace(to_replace = np.nan, value = mean_founded)

In [6]:
# look at the possible values for the job title category
df['job_title'].value_counts()

Data Scientist                                                                       131
Data Engineer                                                                         53
Senior Data Scientist                                                                 34
Data Analyst                                                                          15
Senior Data Engineer                                                                  14
                                                                                    ... 
Senior Research Statistician- Data Scientist                                           1
Senior Machine Learning (ML) Engineer / Data Scientist - Cyber Security Analytics      1
Assistant Director/Director, Office of Data Science                                    1
Customer Data Scientist/Sales Engineer (Bay                                            1
Data Scientist - Bioinformatics                                                        1
Name: job_title, Leng

In [7]:
# parse the city out from the location column and save it in a new column called 'city'
df['city'] = df['location'].apply(lambda x: x.split(',')[0])
# list of all the cities that have at least 5 job postings in the dataset (35 cities)
cities_5_jobs = df.city.value_counts()[0:35].keys()
# label the top 35 cities as the city, and any others as 'other'
df['city'] = df['city'].apply(lambda x: x if x in cities_5_jobs else 'other')

# perform some consolidation of cities based on close geographic region
# ex: Mountain View, Palo Alto, and San Jose are all part of the Bay Area region, so consolidate these into 1 category
consolidated_city_dict = {'South San Francisco': 'San Francisco', 
                          'Mountain View': 'Bay Area', 'Palo Alto': 'Bay Area', 'Washington': 'DC',
                          'Cambridge': 'Boston', 'San Jose': 'Bay Area'}

df['city'] = df['city'].replace(consolidated_city_dict)
# show the consolidated list of top cities with at least 5 job postings
df['city'].value_counts()

other             309
Boston             71
San Francisco      57
New York           55
Chicago            32
Bay Area           28
Pittsburgh         12
Rockville          11
DC                 11
Richland           10
Herndon            10
Springfield        10
Winston-Salem      10
Indianapolis        9
San Diego           9
Austin              8
Rochester           7
Marlborough         6
Gaithersburg        6
Dallas              6
Huntsville          6
Charlotte           6
Phoenix             6
Chantilly           6
Salt Lake City      6
Cincinnati          5
Denver              5
Philadelphia        5
Seattle             5
Nashville           5
Worcester           5
Milwaukee           5
Name: city, dtype: int64

In [8]:
# since there are many job titles, consolidate these into a smaller subset of job families in the data realm
def role_abstractor(job_title):
    if 'data scientist' in job_title.lower():
        return 'data scientist'
    elif 'data engineer' in job_title.lower():
        return 'data engineer'
    elif 'analyst' in job_title.lower() or \
        'business intelligence' in job_title.lower() or \
        'bi' in job_title.lower():
        return 'data analyst'
    elif 'research' in job_title.lower() or 'R&D' in job_title.lower():
        return 'research scientist'
    elif 'machine learning' in job_title.lower():
        return 'machine learning engineer'
    elif 'manager' in job_title.lower():
        return 'manager'
    elif 'director' in job_title.lower():
        return 'director'
    elif 'product' in job_title.lower():
        return 'product analyst'
    elif 'analytics' in job_title.lower():
        return 'data analyst'
    elif 'software' in job_title.lower():
        return 'software engineer'
    else:
        return 'other'

df['role'] = df['job_title'].apply(role_abstractor)
# view the list and count of each new job family
df['role'].value_counts()

data scientist               279
data analyst                 143
other                        121
data engineer                119
research scientist            22
manager                       20
machine learning engineer     15
director                      13
product analyst                6
software engineer              4
Name: role, dtype: int64

In [9]:
# seniority could also play a factor in the salary, so let's try to infer the seniority of the employee based on their job title

# These words in the job title infer entry level employees or low seniority
low_sr = ['associate', 'jr', 'I', 'entry', 'junior']
# these words in the job title infer mid career employees or medium seniority
med_sr = ['senior', 'II', 'III', 'lead', 'head', 'sr']
# these words in the job title infer lots of experience in the field and high seniority
high_sr = ['staff', 'principal', 'director', 'chief']

# label each job title as low, medium, or high seniority based on if the title contains the key words in the lists above
def find_seniority(job_title):
    title_text = job_title.lower()
    # 3 will represent high seniority
    if any(ele in title_text for ele in high_sr):
        return 3
    # 2 will represent medium seniority
    elif any(ele in title_text for ele in med_sr):
        return 2
    # 1 will represent low seniority
    elif any(ele in title_text for ele in low_sr):
        return 1
    # if the title doesn't contain any of the key words, label it as medium seniority (2)
    else:
        return 2
    

df['seniority'] = df['job_title'].apply(find_seniority)
# look at the count of each level of seniority
df['seniority'].value_counts()    

2    654
3     63
1     25
Name: seniority, dtype: int64

In [10]:
WORK_HOURS_PER_YEAR = 2080

# function to find the average salary of an employee based on the range and type of salary listed
def find_avg_salary(string_salary):
    char_list = ['$', 'k', 'K', ' ', 'P', 'e', 'r', 'H', 'o', 'u', 'r']
    
    # if the salary is in a weird format: "Employer Provided Salary:$x-$y"
    if "Employer" in string_salary[0:8]:
        # if the salary is listed as hourly instead of annual, convert the hourly range to an annual salary
        if 'hour' in string_salary.lower():
            str_list = string_salary.split(':')
            salary_range = str_list[1]
            # remove unnecessary characters in the string
            salary_range=re.sub("|".join(char_list), "", salary_range)
            salary_range = salary_range.replace('$', '')
            salary_range_split = salary_range.split('-')
            # convert the hourly rate to annual salary in thousands
            salary_range_split[0] = (int(salary_range_split[0]) * WORK_HOURS_PER_YEAR)/1000
            salary_range_split[1] = (int(salary_range_split[1]) * WORK_HOURS_PER_YEAR)/1000
        
        # if the salary is listed as annual
        else:
            # strip unnecessary characters
            str_list = string_salary.split(':')
            salary_range = str_list[1]
            salary_range=re.sub("|".join(char_list), "", salary_range)
            salary_range = salary_range.replace('$', '')
            salary_range_split = salary_range.split('-')
    
    # if the data is in the default salary format "$x-$y (Glassdoor est.)" 
    else:
        # if the salary data is in an hourly format
        if 'hour' in string_salary.lower():
            str_list = string_salary.split('Per')
            salary_range = str_list[0]
            # strip unnecessary characters
            salary_range=re.sub("|".join(char_list), "", salary_range)
            salary_range = salary_range.replace('$', '')
            salary_range_split = salary_range.split('-')
            # convert the hourly rate to an annual salary in thousands
            salary_range_split[0] = (int(salary_range_split[0]) * WORK_HOURS_PER_YEAR)/1000
            salary_range_split[1] = (int(salary_range_split[1]) * WORK_HOURS_PER_YEAR)/1000
        
        # if the salary is in the annual format
        else:
            
            str_list = string_salary.split('(')
            salary_range = str_list[0]
            # strip unnecessary characters
            salary_range=re.sub("|".join(char_list), "", salary_range)
            salary_range = salary_range.replace('$', '')
            salary_range_split = salary_range.split('-')
    # compute the average salary by taking the average of the upper and lower bounds of the given range
    avg_salary = (int(salary_range_split[0]) + int(salary_range_split[1])) * 0.5
    return avg_salary



In [11]:
# apply the function to the dataframe column
df['avg_salary'] = df['salary'].apply(find_avg_salary)
df.head()

,job_title,salary,job_description,company_rating,company_name,location,headquarters,company_size,founded,ownership_type,industry,sector,revenue,competitors,city,role,seniority,avg_salary
1,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973.0,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,other,data scientist,2,72.0
2,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984.0,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,other,data scientist,2,87.5
3,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010.0,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,other,data scientist,2,85.0
4,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965.0,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",Richland,data scientist,2,76.5
5,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998.0,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",New York,data scientist,2,114.5


In [12]:
# grab the company name text from the company name column (neglect the rating)
df['company_name'] = df['company_name'].apply(lambda x: x.split('\n')[0])

# create a new column called state that takes the state abbreviation from the location column
df['state'] = df['location'].apply(lambda x: x.split(',')[1].strip(' '))
df.head()

,job_title,salary,job_description,company_rating,company_name,location,headquarters,company_size,founded,ownership_type,industry,sector,revenue,competitors,city,role,seniority,avg_salary,state
1,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973.0,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,other,data scientist,2,72.0,NM
2,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System,"Linthicum, MD","Baltimore, MD",10000+ employees,1984.0,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,other,data scientist,2,87.5,MD
3,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010.0,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,other,data scientist,2,85.0,FL
4,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL,"Richland, WA","Richland, WA",1001 to 5000 employees,1965.0,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",Richland,data scientist,2,76.5,WA
5,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,1998.0,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",New York,data scientist,2,114.5,NY


In [13]:
# look at the summary stats for the numeric columns of the data
df.describe()

,company_rating,founded,seniority,avg_salary
count,742.000000,742.000000,742.000000,742.000000
mean,3.688372,1969.969653,2.051213,101.484501
std,0.566104,52.233003,0.340782,37.482449
min,1.900000,1744.000000,1.000000,15.500000
25%,3.300000,1961.000000,2.000000,73.500000
50%,3.700000,1988.000000,2.000000,97.500000
75%,4.000000,2007.000000,2.000000,122.500000
max,5.000000,2019.000000,3.000000,254.000000


In [14]:
# look at the company size values (looks like there are some values missing)
df['company_size'].value_counts()

1001 to 5000 employees     150
501 to 1000 employees      134
10000+ employees           130
201 to 500 employees       117
51 to 200 employees         94
5001 to 10000 employees     76
1 to 50 employees           31
Unknown                      9
-1                           1
Name: company_size, dtype: int64

In [15]:
# group the company size into either small, medium, large, or missing categories
def company_size_classification(text):
    # small businesses characterized as <200 employees
    if text in ['1 to 50 employees', '51 to 200 employees']:
        return 'small'
    # medium businesses classified as 200-1000 employees
    elif text in ['201 to 500 employees', '501 to 1000 employees']:
        return 'medium'
    # large businesses classified as 1000+ employees
    elif text in ['1001 to 5000 employees','5001 to 10000 employees', '10000+ employees']:
        return 'large'
    else:
        return 'N/A'
df['company_size'] = df['company_size'].apply(company_size_classification)


In [16]:
df.head()

,job_title,salary,job_description,company_rating,company_name,location,headquarters,company_size,founded,ownership_type,industry,sector,revenue,competitors,city,role,seniority,avg_salary,state
1,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research,"Albuquerque, NM","Goleta, CA",medium,1973.0,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,other,data scientist,2,72.0,NM
2,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System,"Linthicum, MD","Baltimore, MD",large,1984.0,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,other,data scientist,2,87.5,MD
3,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4,"Clearwater, FL","Clearwater, FL",medium,2010.0,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,other,data scientist,2,85.0,FL
4,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL,"Richland, WA","Richland, WA",large,1965.0,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",Richland,data scientist,2,76.5,WA
5,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",small,1998.0,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",New York,data scientist,2,114.5,NY


In [17]:
# look at the frequency of each sector in the data
df['sector'].value_counts()

Information Technology                180
Biotech & Pharmaceuticals             112
Business Services                      97
Insurance                              69
Health Care                            49
Finance                                42
Manufacturing                          34
Aerospace & Defense                    25
Education                              23
Retail                                 15
Oil, Gas, Energy & Utilities           14
Government                             11
-1                                     10
Non-Profit                              9
Transportation & Logistics              8
Real Estate                             8
Travel & Tourism                        8
Telecommunications                      6
Media                                   6
Arts, Entertainment & Recreation        4
Consumer Services                       4
Construction, Repair & Maintenance      3
Mining & Metals                         3
Agriculture & Forestry            

In [18]:
# this function consolidates the less common sectors into a category called 'other' so we can focus on the main sectors
def sector_aggregator(sector):
    if sector in [np.nan, 'Non-Profit', 'Transportation & Logistics',
                  'Real Estate', 'Travel & Tourism', 'Media', 'Telecommunications',
                  'Arts, Entertainment & Recreation', 'Customer Services', 'Construction',
                  'Repair & Maintenance', 'Mining & Metals', 'Agriculture & Forestry',
                  'Accounting & Legal']:
        return 'other'
    else:
        return sector
    
df['sector'] = df['sector'].apply(sector_aggregator)
df.head()

,job_title,salary,job_description,company_rating,company_name,location,headquarters,company_size,founded,ownership_type,industry,sector,revenue,competitors,city,role,seniority,avg_salary,state
1,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research,"Albuquerque, NM","Goleta, CA",medium,1973.0,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,other,data scientist,2,72.0,NM
2,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System,"Linthicum, MD","Baltimore, MD",large,1984.0,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,other,data scientist,2,87.5,MD
3,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4,"Clearwater, FL","Clearwater, FL",medium,2010.0,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,other,data scientist,2,85.0,FL
4,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL,"Richland, WA","Richland, WA",large,1965.0,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",Richland,data scientist,2,76.5,WA
5,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",small,1998.0,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",New York,data scientist,2,114.5,NY


In [19]:
# calculate the company age by subtracting the founded date from todays date
current_year = date.today().year
df['company_age'] = current_year - df['founded']
df['company_age'].head()



1    47.0
2    36.0
3    10.0
4    55.0
5    22.0
Name: company_age, dtype: float64

In [20]:
# keep only the needed columns in the data for future analysis
df_final = df[['role', 'seniority', 'avg_salary', 'state', 'company_rating', 'company_size', 'sector', 'company_age', 'city']]
df_final.head()

,role,seniority,avg_salary,state,company_rating,company_size,sector,company_age,city
1,data scientist,2,72.0,NM,3.8,medium,Aerospace & Defense,47.0,other
2,data scientist,2,87.5,MD,3.4,large,Health Care,36.0,other
3,data scientist,2,85.0,FL,4.8,medium,Business Services,10.0,other
4,data scientist,2,76.5,WA,3.8,large,"Oil, Gas, Energy & Utilities",55.0,Richland
5,data scientist,2,114.5,NY,2.9,small,Business Services,22.0,New York


In [21]:
# make sure there are no null values in the final dataset
df_final.isnull().sum()

role              0
seniority         0
avg_salary        0
state             0
company_rating    0
company_size      0
sector            0
company_age       0
city              0
dtype: int64

In [22]:
# export the dataset to a csv file
df_final.to_csv("cleaned_data.csv", index = False)